# 1.4 Introduction Linear Regression #

A regression model is a statistical model of the influence of random variables on a certain random variable. If we assume that they follow a simple linear relationship, of the form:

$$
Y = \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_k X_k,
$$

then this is a linear regression model. In this case, there are $k$ *independent variables*. They are alternatively called *explanatory variables*. Sometimes, they are constants, and then they are called *constant terms*. The coefficients $\beta_i$ are the parameters of the model that have to be estimated using data.

#### Simplest Linear Model ####

The simplest cast of a linear model is $Y = \alpha + \beta X$. We will work with some data from Yahoo! Finance subsequently.

For such a simple model, using a scatter plot can aid in visualization.

In [24]:
#!pip install yfinance
#!pip install yahoofinancials
#!pip install pandas
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials

cutoffdata = '2022-11-10' #Only take data after this year

# Download AMEX data, S&P 500 Data

amex_data = yf.download('AXP')
print(amex_data.index)
print(cutoffdata in amex_data.index)
# amex_data[amex_data.apply(lambda x: x['c']%2 == 0, axis = 1)]
# amex_data = amex_data.filter(items)
# amex_data.loc[amex_data['index']>=cutoffdata]
# print(amex_data)
# SnP_data = yf.download('^GSPC')
# print(SnP_data)

[*********************100%***********************]  1 of 1 completed
DatetimeIndex(['1972-06-01', '1972-06-02', '1972-06-05', '1972-06-06',
               '1972-06-07', '1972-06-08', '1972-06-09', '1972-06-12',
               '1972-06-13', '1972-06-14',
               ...
               '2022-11-10', '2022-11-11', '2022-11-14', '2022-11-15',
               '2022-11-16', '2022-11-17', '2022-11-18', '2022-11-21',
               '2022-11-22', '2022-11-23'],
              dtype='datetime64[ns]', name='Date', length=12732, freq=None)
True
